# Paso 1. Generar embeddings

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import pickle
import os
import re
import json
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


print(f"🔥 CUDA disponible: {torch.cuda.is_available()}")
print(f" GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No detectada'}")
print(f" VRAM disponible: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB" if torch.cuda.is_available() else "")

# %%
def clean_text(text):
    """Limpiar texto en español"""
    if pd.isna(text) or text == "":
        return ""
    
    text = str(text)
    # Remover patrón de calificación
    text = re.sub(r'Calificación:\s*[\d\.]+/10\s*-\s*', '', text)
    # Normalizar espacios
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

def extract_rating(text):
    """Extraer calificación del comentario"""
    if pd.isna(text):
        return None
    
    match = re.search(r'Calificación:\s*([\d\.]+)/10', str(text))
    if match:
        try:
            return float(match.group(1))
        except:
            return None
    return None

def create_enriched_text(row):
    """Crear texto enriquecido con contexto"""
    parts = []
    
    # Departamento
    if pd.notna(row['DEPARTAMENTO']) and row['DEPARTAMENTO'] != "":
        dept = str(row['DEPARTAMENTO']).replace('DEPTO. DE ', '').strip()
        parts.append(f"Departamento: {dept}")
    
    # División
    if pd.notna(row['DIVISION']) and row['DIVISION'] != "":
        div = str(row['DIVISION']).replace('División de ', '').strip()
        parts.append(f"División: {div}")
    
    # Materia
    if pd.notna(row['MATERIA']) and str(row['MATERIA']).strip() != "":
        materia = re.sub(r'\([^)]*\)', '', str(row['MATERIA'])).strip()
        if materia:
            parts.append(f"Materia: {materia}")
    
    # Comentario
    comment = clean_text(row['COMENTARIOS'])
    if comment:
        parts.append(f"Comentario: {comment}")
    
    return " | ".join(parts)

print("✅ Funciones de preprocesamiento listas")

# %%
# Configuración de rutas
CSV_PATH = r"# === NOTE: Replace with local path ==="
OUTPUT_DIR = r"# === NOTE: Replace with local path ==="

print(f" CSV: {CSV_PATH}")
print(f" Salida: {OUTPUT_DIR}")

# Crear directorio de salida
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# %%
# Cargar datos
print(" Cargando datos...")
df = pd.read_csv(CSV_PATH, encoding='utf-8')

print(f"✅ Dataset cargado:")
print(f"  • Total filas: {len(df)}")
print(f"  • Profesores únicos: {df['PROFESOR'].nunique()}")
print(f"  • Departamentos: {df['DEPARTAMENTO'].nunique()}")
print(f"  • Comentarios válidos: {df['COMENTARIOS'].notna().sum()}")

# %%
# Preparar datos para embeddings
print("🔧 Preparando datos...")

# Extraer ratings
df['rating'] = df['COMENTARIOS'].apply(extract_rating)

# Crear textos enriquecidos
df['texto_enriquecido'] = df.apply(create_enriched_text, axis=1)

# Filtrar datos útiles
mask = (df['texto_enriquecido'].str.len() > 15) & (df['COMENTARIOS'].notna())
df_clean = df[mask].copy()

print(f"✅ Datos preparados:")
print(f"  • Filas útiles: {len(df_clean)}")
print(f"  • Ratings extraídos: {df_clean['rating'].notna().sum()}")
print(f"  • Rating promedio: {df_clean['rating'].mean():.2f}")

# Mostrar ejemplos
print(f"\n📝 Ejemplos de textos enriquecidos:")
for i in range(min(3, len(df_clean))):
    text = df_clean.iloc[i]['texto_enriquecido']
    print(f"  {i+1}. {text[:120]}...")

# %%
# Modelos disponibles optimizados para español
MODELS = {
    'spanish_specialized': 'hiiamsid/sentence_transformers_spanish',
    'multilingual_robust': 'paraphrase-multilingual-MiniLM-L12-v2',
    'academic_optimized': 'distiluse-base-multilingual-cased'
}

# Seleccionar modelo (cambia aquí si quieres)
MODEL_CHOICE = 'multilingual_robust'  # Recomendado para español latinoamericano
model_name = MODELS[MODEL_CHOICE]

print(f" Cargando modelo: {model_name}")

#
# Cargar modelo con optimización GPU
try:
    model = SentenceTransformer(model_name)
    
    # Configurar para RTX 3050
    if torch.cuda.is_available():
        model = model.cuda()
        torch.cuda.empty_cache()  # Limpiar VRAM
        
    else:
        print(" Usando CPU")
    
    print(f" Modelo '{MODEL_CHOICE}' listo")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("🔄 Probando modelo alternativo...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    MODEL_CHOICE = 'multilingual_robust'


# Generar embeddings con optimización GPU
print(f"🚀 Generando embeddings para {len(df_clean)} textos...")

texts = df_clean['texto_enriquecido'].tolist()

# Batch size optimizado 
BATCH_SIZE = 32  # Ajusta si necesitas (16 para más seguridad, 64 para más velocidad)

try:
    embeddings = model.encode(
        texts,
        batch_size=BATCH_SIZE,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    print(f" Embeddings generados:")
    print(f"  • Dimensiones: {embeddings.shape}")
    print(f"  • Memoria usada: {embeddings.nbytes / 1024**2:.1f} MB")
    print(f"  • Normalizado: Sí")
    
except Exception as e:
    print(f"❌ Error: {e}")
    embeddings = None

# %%
# Crear estructura de datos completa
if embeddings is not None:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    embeddings_package = {
        'embeddings': embeddings,
        'metadata': {
            'model_choice': MODEL_CHOICE,
            'model_name': model_name,
            'embedding_dim': embeddings.shape[1],
            'num_samples': embeddings.shape[0],
            'batch_size': BATCH_SIZE,
            'normalized': True,
            'device_used': 'cuda' if torch.cuda.is_available() else 'cpu',
            'generation_date': datetime.now().isoformat()
        },
        'data': {
            'professors': df_clean['PROFESOR'].tolist(),
            'departments': df_clean['DEPARTAMENTO'].tolist(),
            'divisions': df_clean['DIVISION'].tolist(),
            'subjects': df_clean['MATERIA'].tolist(),
            'ratings': df_clean['rating'].tolist(),
            'original_comments': df_clean['COMENTARIOS'].tolist(),
            'enriched_texts': texts
        }
    }
    
    print(" Paquete de datos creado")

# %%
# Guardar embeddings
if embeddings is not None:
    
    # Crear nombres de archivos
    base_name = f"profesores_embeddings_{MODEL_CHOICE}_{timestamp}"
    
    # 1. Embeddings puros (numpy) - Para cargar rápido
    embeddings_file = os.path.join(OUTPUT_DIR, f"{base_name}.npy")
    np.save(embeddings_file, embeddings)
    
    # 2. Paquete completo (pickle) - Para análisis completo
    complete_file = os.path.join(OUTPUT_DIR, f"{base_name}_complete.pkl")
    with open(complete_file, 'wb') as f:
        pickle.dump(embeddings_package, f)
    
    # 3. Metadatos (JSON) - Para referencia rápida
    metadata_file = os.path.join(OUTPUT_DIR, f"{base_name}_metadata.json")
    with open(metadata_file, 'w', encoding='utf-8') as f:
        metadata_summary = {
            'modelo': embeddings_package['metadata'],
            'resumen_datos': {
                'profesores_unicos': len(set(embeddings_package['data']['professors'])),
                'departamentos': len(set(filter(pd.notna, embeddings_package['data']['departments']))),
                'divisiones': len(set(filter(pd.notna, embeddings_package['data']['divisions']))),
                'rating_promedio': np.nanmean([r for r in embeddings_package['data']['ratings'] if r is not None]),
                'total_muestras': len(embeddings_package['data']['professors'])
            }
        }
        json.dump(metadata_summary, f, indent=2, ensure_ascii=False)
    
    print(f"\n Archivos guardados:")
    print(f"  • Embeddings: {embeddings_file}")
    print(f"  • Datos completos: {complete_file}")
    print(f"  • Metadatos: {metadata_file}")

# %%
# Análisis de embeddings generados
if embeddings is not None:
    print(f"\n Análisis de embeddings:")
    print(f"  • Forma: {embeddings.shape}")
    print(f"  • Dimensión: {embeddings.shape[1]}")
    print(f"  • Samples: {embeddings.shape[0]}")
    print(f"  • Rango valores: [{embeddings.min():.4f}, {embeddings.max():.4f}]")
    print(f"  • Media: {embeddings.mean():.4f}")
    print(f"  • Std: {embeddings.std():.4f}")
    
    # Verificar normalización
    norms = np.linalg.norm(embeddings, axis=1)
    print(f"  • Normas L2: min={norms.min():.4f}, max={norms.max():.4f}")
    
    # Estadísticas por departamento
    dept_stats = df_clean.groupby('DEPARTAMENTO').agg({
        'rating': ['count', 'mean'],
        'PROFESOR': 'nunique'
    }).round(2)
    
    print(f"\n📊 Estadísticas por departamento:")
    print(dept_stats.head())

# %%
print("\n🎉 ¡Embeddings generados exitosamente!")
print(" Archivos listos en:", OUTPUT_DIR)
print("\n Próximos pasos:")
print("  1. ✅ Embeddings base generados")
print("  2. 🔄 Siguiente: Red neuronal con attention")

# %%
# Función para cargar embeddings después (para uso futuro)
def load_embeddings(embeddings_dir, timestamp=None):
    """
    Función para cargar embeddings generados
    Si timestamp=None, carga el más reciente
    """
    files = list(Path(embeddings_dir).glob("*_complete.pkl"))
    if not files:
        print(" No se encontraron archivos de embeddings")
        return None
    
    if timestamp:
        target_file = [f for f in files if timestamp in str(f)]
        if target_file:
            file_to_load = target_file[0]
        else:
            print(f" No se encontró archivo con timestamp {timestamp}")
            return None
    else:
        # Cargar el más reciente
        file_to_load = max(files, key=os.path.getctime)
    
    print(f" Cargando: {file_to_load}")
    
    with open(file_to_load, 'rb') as f:
        data = pickle.load(f)
    
    print(f" Embeddings cargados: {data['embeddings'].shape}")
    return data

print(" Función de carga definida para uso futuro")